In [1]:
import trimesh 
import xatlas


In [2]:
import numpy as np
import trimesh
file_path = "/home/zqq/lagrangian_gaussian_splatting/Simulation/Codim-IPC/Projects/FEMShell/input/actorhq_smplx_2/shell0.obj"
mesh = trimesh.load(file_path)
file_path_cloth = "/home/zqq/lagrangian_gaussian_splatting/Simulation/Codim-IPC/Projects/FEMShell/input/a_skirt_new/dress.obj"
mesh_2 = trimesh.load(file_path_cloth)
vertices_cloth =  mesh_2.vertices
cloth_vertice_num = vertices_cloth.shape[0]
face_cloth = mesh_2.faces
vertices = mesh.vertices
faces = mesh.faces


import open3d
mesh_o3d = open3d.geometry.TriangleMesh()
mesh_o3d.vertices = open3d.utility.Vector3dVector(vertices_cloth)
mesh_o3d.triangles = open3d.utility.Vector3iVector(face_cloth)
triangle_clusters, cluster_n_triangles, cluster_area = mesh_o3d.cluster_connected_triangles()
max_cluster = np.argmax(cluster_area)
mesh_o3d.remove_triangles_by_mask(np.array(triangle_clusters) != max_cluster)
vertices_cloth = np.array(mesh_o3d.vertices)
face_cloth = np.array(mesh_o3d.triangles)

face_total = []
vertices_total = []
for elem in vertices_cloth:
    vertices_total.append(elem)
for elem in vertices:
    vertices_total.append(elem)

for elem in face_cloth:
    face_total.append([elm for elm in elem])
for elem in faces:
    face_total.append([i+cloth_vertice_num for i in elem])
total_mesh = trimesh.Trimesh(vertices=np.array(vertices_total), faces=np.array(face_total))
path = "/home/zqq/lagrangian_gaussian_splatting/Simulation/Codim-IPC/Projects/FEMShell/input/a_skirt_new/drape.obj"
total_mesh.export(path)


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


'# https://github.com/mikedh/trimesh\nv 0.04345000 0.60837600 -0.16032700\nv 0.05206600 0.61150800 -0.16930500\nv 0.06300500 0.61430000 -0.17374400\nv 0.07418900 0.61704500 -0.17728300\nv 0.08509900 0.61987400 -0.18178100\nv 0.09497300 0.62286200 -0.18879100\nv 0.09997600 0.62577100 -0.20112100\nv 0.09429700 0.62708600 -0.21346100\nv 0.08306300 0.62706000 -0.21821400\nv 0.07113900 0.62675400 -0.21817000\nv 0.05923200 0.62642500 -0.21740900\nv 0.04731500 0.62611600 -0.21686900\nv 0.03539300 0.62582300 -0.21648800\nv 0.02346500 0.62554600 -0.21627300\nv 0.01153800 0.62528800 -0.21623500\nv -0.00038800 0.62505500 -0.21643600\nv -0.01231100 0.62483900 -0.21684100\nv -0.02422500 0.62464400 -0.21747500\nv -0.03613500 0.62446900 -0.21825200\nv -0.04802200 0.62420200 -0.21714800\nv -0.05757500 0.62327100 -0.20875600\nv -0.05794200 0.62094900 -0.19490400\nv -0.05171100 0.61849600 -0.18322500\nv -0.04374400 0.61615000 -0.17309900\nv -0.03521400 0.61387400 -0.16360900\nv -0.02681900 0.61162500 -0

In [ ]:
file_path = "/home/zqq/lagrangian_gaussian_splatting/Simulation/Codim-IPC/Projects/FEMShell/input/dress_knife/submesh.obj"
mesh = trimesh.load(file_path)
vertices = mesh.vertices
faces = mesh.faces

mesh = trimesh.Trimesh(vertices=vertices, faces=faces)
path = "/home/zqq/lagrangian_gaussian_splatting/Simulation/Codim-IPC/Projects/FEMShell/input/dress_knife/submesh3.obj"
mesh.export(path, file_type="obj")

In [ ]:
# import numpy as np
# vmapping, indices, uvs = xatlas.parametrize(mesh.vertices, mesh.faces)
# vmapping_unique, order = np.unique(vmapping,return_index=True)
# xatlas.export(path, mesh.vertices[vmapping], indices, uvs)

In [ ]:
import pymeshlab
ms = pymeshlab.MeshSet()
ms.load_new_mesh(file_path)
# not familiar with the crop API, but I'm sure it's doable
# now we generate UV map; there are a couple options here but this is a naive way
ms.compute_texcoord_by_function_per_vertex()
# create texture using UV map and vertex colors
# ms.compute_texmap_from_color(textname=f"my_texture_name") # textname will be filename of a png, should not be a full path
# texture file won't be saved until you save the mesh
# ms.save_current_mesh(mesh_path)
#export uv map
ms.save_current_mesh(path)
#load path and remove any line start with vn 
with open(path, 'r') as f:
    lines = f.readlines()
    new_lines = []
    for line in lines:
        if line.startswith('vn'):
            continue
        if line.startswith('f '):
            face = line.split(" ")[1:]
            face = [int(i.split("/")[0]) for i in face]
            line = "f "+str(face[0]) + "/" + str(face[0])+" " + str(face[1]) + "/" + str(face[1])+" " + str(face[2]) + "/" + str(face[2])+ "\n"
            print(line)
        new_lines.append(line)
#write to file
with open(path, 'w') as f:
    f.writelines(new_lines)


In [ ]:
import numpy as np
path = "/home/zqq/lagrangian_gaussian_splatting/Simulation/Codim-IPC/Projects/FEMShell/input/dress_knife/submesh3.obj"
mesh_cloth = trimesh.load(path)
print(mesh_cloth.vertices.shape,mesh_cloth.faces.shape)
path = "/home/zqq/lagrangian_gaussian_splatting/Simulation/Codim-IPC/Projects/FEMShell/input/Rumba_Dancing/shell0.obj"
mesh_body = trimesh.load(path)
#combine two into one 
vertices = np.concatenate((mesh_cloth.vertices, mesh_body.vertices), axis=0)
faces = np.concatenate((mesh_cloth.faces, mesh_body.faces+len(mesh_cloth.vertices)), axis=0)
print(vertices.shape,faces.shape)
mesh_total = trimesh.Trimesh(vertices=vertices, faces=faces)
path = "/home/zqq/lagrangian_gaussian_splatting/Simulation/Codim-IPC/Projects/FEMShell/input/dress_knife/drape3.obj"
mesh_total.export(path)

In [ ]:
mesh = trimesh.load(path)
print(mesh.vertices.shape,mesh.faces.shape)

In [ ]:
vmapping_sort = np.argsort(np.array(vmapping_unique))

In [ ]:
order

In [ ]:
 #load file
# if file line starts with "f", turn it from 
# f num3 num2 num1
# # to 
# # f num3/num3 num2/num2 num1/num1
# # else just copy it
# # save to new file

# with open(path, "r") as f:
#     lines = f.readlines()
#     new_lines = []
#     for line in lines:
#         if line[0] == "f":
#             #load faces
#             face = line.split(" ")[1:]
#             new_face = "f "
#             for f in face:
#                 f = int(f)
#                 new_face +=  str(f) + "/" + str(f) + " "
#             print(new_face)
#             new_face += "\n"
#             new_lines.append(new_face)
#         else:
#             print(line)
#             new_lines.append(line)

# #save to new file
# with open("/home/zqq/lagrangian_gaussian_splatting/Simulation/Codim-IPC/Projects/FEMShell/input/dress_knife/submesh2.obj", "w") as f:
#     for line in new_lines:
#         print(line)
#         f.write(line)

#add vt 
import numpy as np
vertices = mesh.vertices
uv = np.linspace(0, 1, len(vertices))
lines = []
for line in uv:
    lines.append("vt " + str(line) + " " + str(line) + "\n")

with open("/home/zqq/lagrangian_gaussian_splatting/Simulation/Codim-IPC/Projects/FEMShell/input/dress_knife/stage.obj", "a") as f:
    for line in lines:
        f.write(line)
        

In [ ]:
#get sebmesh
vertices = mesh.vertices
faces = mesh.faces
print(vertices.shape)
print(faces.shape)


In [ ]:
file_path = "/home/zqq/lagrangian_gaussian_splatting/Simulation/Codim-IPC/Projects/FEMShell/input/dress_knife/drape.obj"
#load file_path line by line 
mesh_stage = trimesh.load("/home/zqq/lagrangian_gaussian_splatting/Simulation/Codim-IPC/Projects/FEMShell/input/dress_knife/stage.obj")
length = mesh_stage.vertices.shape[0] + 1
print("length",length)
with open(file_path, "r") as f:
    lines = f.readlines()
    new_vertices = []
    new_faces = []
    for line in lines:
        if line[0] == "v":
            if len(new_vertices)<length:
                new_vertices.append(line)
        
            
        if line[0] == "f":
            face = line.split(" ")[1:]
            new_face = "f "
            for f in face:
                f = f.split("/")[0]
                f = int(f)
                if f < length:
                    new_face +=  str(f) + " "
                else:
                    pass 
            new_face += "\n"
            if new_face != "f \n":
                new_faces.append(new_face)
print(new_faces[-4:])

#write to new file
with open("/home/zqq/lagrangian_gaussian_splatting/Simulation/Codim-IPC/Projects/FEMShell/input/dress_knife/submesh_3d.obj", "w") as f:
    for line in new_vertices:
        f.write(line)
    for line in new_faces:
        f.write(line)

In [ ]:
file_path = "/home/zqq/lagrangian_gaussian_splatting/Simulation/Codim-IPC/Projects/FEMShell/input/dress_knife/drape.obj"
#load file_path line by line 
mesh_stage = trimesh.load("/home/zqq/lagrangian_gaussian_splatting/Simulation/Codim-IPC/Projects/FEMShell/input/dress_knife/stage.obj")
length = mesh_stage.vertices.shape[0] + 1
print("length",length)
with open(file_path, "r") as f:
    lines = f.readlines()
    new_vertices = []
    new_faces = []
    for line in lines:
        if line[0] == "v":
            if len(new_vertices)<length:
                verticce = line.split(" ")[1:]
                verticce = str(float(verticce[0])) + " " + str(float(verticce[1])-0.1) + " " + str(float(verticce[2])) + "\n"
                line = "v " + verticce
                new_vertices.append(line)
            else:
                new_vertices.append(line)
        
            
        if line[0] == "f":
            new_faces.append(line)
print(new_faces[-4:])

#write to new file
with open("/home/zqq/lagrangian_gaussian_splatting/Simulation/Codim-IPC/Projects/FEMShell/input/dress_knife/drage_shift.obj", "w") as f:
    for line in new_vertices:
        f.write(line)
    for line in new_faces:
        f.write(line)

In [ ]:
file_path = "/home/zqq/lagrangian_gaussian_splatting/Simulation/Codim-IPC/Projects/FEMShell/input/dress_knife/stage.obj"
#load file_path line by line 
mesh_stage = trimesh.load("/home/zqq/lagrangian_gaussian_splatting/Simulation/Codim-IPC/Projects/FEMShell/input/dress_knife/stage.obj")
length = mesh_stage.vertices.shape[0] + 1
print("length",length)
with open(file_path, "r") as f:
    lines = f.readlines()
    new_vertices = []
    new_faces = []
    for line in lines:
        if line[0] == "v":
            if len(new_vertices)<length:
                new_vertices.append(line)
        
            
        if line[0] == "f":
            face = line.split(" ")[1:]
            new_face = "f "
            for f in face:
                f = f.split("/")[0]
                f = int(f)
                if f < length:
                    new_face +=  str(f) + " "
                else:
                    pass 
            new_face += "\n"
            if new_face != "f \n":
                new_faces.append(new_face)
print(new_faces[-4:])

#write to new file
with open("/home/zqq/lagrangian_gaussian_splatting/Simulation/Codim-IPC/Projects/FEMShell/input/dress_knife/submesh.obj", "w") as f:
    for line in new_vertices:
        f.write(line)
    for line in new_faces:
        f.write(line)

In [ ]:
file_path = "/home/zqq/lagrangian_gaussian_splatting/Simulation/Codim-IPC/Projects/FEMShell/input/dress_knife/drape.obj"
#load file_path line by line 
mesh_stage = trimesh.load("/home/zqq/lagrangian_gaussian_splatting/Simulation/Codim-IPC/Projects/FEMShell/input/dress_knife/stage.obj")
length = mesh_stage.vertices.shape[0]
print("length",length)
with open(file_path, "r") as f:
    lines = f.readlines()
    new_vertices = []
    new_faces = []
    for line in lines:
        if line[0] == "v":
            if len(new_vertices)<length:
                new_vertices.append(line)
        
            
        if line[0] == "f":
            face = line.split(" ")[1:]
            new_face = "f "
            for f in face:
                f = int(f)
                if f < length:
                    new_face +=  str(f) + " "
                else:
                    pass 
            new_face += "\n"
            if new_face != "f \n":
                new_faces.append(new_face)
print(new_faces)

#write to new file
with open("/home/zqq/lagrangian_gaussian_splatting/Simulation/Codim-IPC/Projects/FEMShell/input/dress_knife/submesh.obj", "w") as f:
    for line in new_vertices:
        f.write(line)
    for line in new_faces:
        f.write(line)

In [7]:
file_path = "/home/zqq/lagrangian_gaussian_splatting/Simulation/Codim-IPC/Projects/FEMShell/input/a_skirt/dress.obj"
with open(file_path, "r") as f:
    lines = f.readlines()
    new_vertices = []
    new_faces = []
    for line in lines:
        if line[0] == "v":
            new_vertices.append(line)
        if line[0] == "f":
            new_faces.append(line)
print(new_faces)
total_vertices = len(new_vertices)

avatar_path = "/home/zqq/lagrangian_gaussian_splatting/Simulation/Codim-IPC/Projects/FEMShell/input/actorhq_smplx/shell0.obj"
with open(avatar_path, "r") as f:
    lines = f.readlines()
    for line in lines:
        if line[0] == "v":
            new_vertices.append(line)
    
        if line[0] == "f":
            face = line.split(" ")[1:]
            new_face = "f "
            for f in face:
                f = int(f) + total_vertices
                new_face +=  str(f) + " "
            
            new_face += "\n"
            if new_face != "f \n":
                new_faces.append(new_face)
print(new_faces)

#write to new file
with open("/home/zqq/lagrangian_gaussian_splatting/Simulation/Codim-IPC/Projects/FEMShell/input/a_skirt/drape.obj", "w") as f:
    for line in new_vertices:
        f.write(line)
    for line in new_faces:
        f.write(line)

['f 185 1 2\n', 'f 1 5779 5778\n', 'f 185 254 146\n', 'f 503 504 308\n', 'f 582 167 232\n', 'f 254 2 3\n', 'f 232 167 5782\n', 'f 127 222 167\n', 'f 5785 5784 168\n', 'f 135 180 349\n', 'f 5786 243 180\n', 'f 5784 5783 222\n', 'f 4 219 159\n', 'f 8 9 612\n', 'f 265 219 5\n', 'f 610 206 152\n', 'f 610 13 14\n', 'f 639 641 223\n', 'f 239 171 12\n', 'f 5789 174 5790\n', 'f 171 132 288\n', 'f 372 5788 5787\n', 'f 154 612 209\n', 'f 223 160 7\n', 'f 784 240 785\n', 'f 5789 240 174\n', 'f 219 307 159\n', 'f 608 5792 5791\n', 'f 161 5797 5796\n', 'f 5795 260 5796\n', 'f 205 260 5795\n', 'f 5792 204 5793\n', 'f 151 609 204\n', 'f 607 5799 5798\n', 'f 118 741 394\n', 'f 5800 181 5801\n', 'f 182 5804 5803\n', 'f 5801 251 5802\n', 'f 5797 161 224\n', 'f 5791 5790 241\n', 'f 170 5806 5805\n', 'f 184 526 790\n', 'f 5809 218 5810\n', 'f 184 5812 5811\n', 'f 91 374 668\n', 'f 165 229 62\n', 'f 5814 253 5815\n', 'f 5816 5815 500\n', 'f 441 5813 5812\n', 'f 280 374 666\n', 'f 75 5817 5816\n', 'f 666 44

In [3]:
path = "/home/zqq/lagrangian_gaussian_splatting/Simulation/Codim-IPC/Projects/FEMShell/input/dress_knife/drape.obj"
mesh = trimesh.load(path)
print(mesh.vertices.max(axis=0),mesh.vertices.min(axis=0))
path = "/home/zqq/lagrangian_gaussian_splatting/Simulation/Codim-IPC/Projects/FEMShell/input/Rumba_Dancing/shell0.obj"
mesh = trimesh.load(path)
print(mesh.vertices.max(axis=0),mesh.vertices.min(axis=0))


[0.641  0.7493 0.132 ] [-0.641     -0.75      -0.1337156]
[0.641  1.4993 0.132 ] [-0.641   0.     -0.1324]


In [ ]:

import trimesh 
path = "/home/zqq/lagrangian_gaussian_splatting/Simulation/Codim-IPC/Projects/FEMShell/input/a_skirt/drape.obj"
mesh = trimesh.load(path)
max_ = mesh.vertices.max(axis=0)
min_ = mesh.vertices.min(axis=0)
shift = (max_ + min_)/2
print(mesh.vertices.max(axis=0))
print(mesh.vertices.min(axis=0))
mesh_vertices = mesh.vertices
mesh_vertices[:,1] -= shift[1]
print(mesh_vertices.max(axis=0))
print(mesh_vertices.min(axis=0))
new_mesh = trimesh.Trimesh(vertices=mesh_vertices, faces=mesh.faces)
#export 
new_mesh.export(path)

In [ ]:
import numpy as np
import trimesh
file_path = "/home/zqq/lagrangian_gaussian_splatting/Simulation/Codim-IPC/Projects/FEMShell/input/actorhq_smplx/shell0.obj"
mesh = trimesh.load(file_path)
file_path_cloth = "/home/zqq/lagrangian_gaussian_splatting/Simulation/Codim-IPC/Projects/FEMShell/input/a_skirt_uv/dress_uv.obj"
mesh_2 = trimesh.load(file_path_cloth)
vertices_cloth =  mesh_2.vertices
cloth_vertice_num = vertices_cloth.shape[0]
face_cloth = mesh_2.faces
vertices = mesh.vertices
faces = mesh.faces


import open3d
mesh_o3d = open3d.geometry.TriangleMesh()
mesh_o3d.vertices = open3d.utility.Vector3dVector(vertices_cloth)
mesh_o3d.triangles = open3d.utility.Vector3iVector(face_cloth)
triangle_clusters, cluster_n_triangles, cluster_area = mesh_o3d.cluster_connected_triangles()
max_cluster = np.argmax(cluster_area)
mesh_o3d.remove_triangles_by_mask(np.array(triangle_clusters) != max_cluster)
vertices_cloth = np.array(mesh_o3d.vertices)
face_cloth = np.array(mesh_o3d.triangles)

face_total = []
vertices_total = []
for elem in vertices_cloth:
    vertices_total.append(elem)
for elem in vertices:
    vertices_total.append(elem)

for elem in face_cloth:
    face_total.append([elm for elm in elem])
for elem in faces:
    face_total.append([i+cloth_vertice_num for i in elem])
total_mesh = trimesh.Trimesh(vertices=np.array(vertices_total), faces=np.array(face_total))
total_mesh.export("/home/zqq/lagrangian_gaussian_splatting/Simulation/Codim-IPC/Projects/FEMShell/input/a_skirt_uv/drape.obj")

In [6]:

import trimesh 
path = "/home/zqq/lagrangian_gaussian_splatting/Simulation/Codim-IPC/Projects/FEMShell/input/a_skirt/drape.obj"
mesh = trimesh.load(path)
min_ = mesh.vertices.min(axis=0)
max_ = mesh.vertices.max(axis=0)
print(min_,max_)
# mesh_vertices = mesh.vertices
# mesh_vertices[:,1] -= min_[1]
# mesh_vertices[:,1] -= 0.75
# print(mesh_vertices.min(axis=0))
# mesh = trimesh.Trimesh(vertices=mesh_vertices, faces=mesh.faces)
# #export
# mesh.export(path)

IndexError: index 24494 is out of bounds for axis 0 with size 17045

In [ ]:
path = glob("/home/zqq/lagrangian_gaussian_splatting/Simulation/Codim-IPC/Projects/FEMShell/input/actorhq_smplx/shell*.obj")
mesh = trimesh.load(path[0])
max_avatar = mesh.vertices.max(axis=0)
min_avatar = mesh.vertices.min(axis=0)
print(max_avatar,min_avatar)

In [2]:

import trimesh 
from glob import glob
path = glob("/home/zqq/lagrangian_gaussian_splatting/Simulation/Codim-IPC/Projects/FEMShell/input/actorhq_smplx/shell*.obj")
for idx,path in enumerate(path):
    if idx == 0:
        mesh = trimesh.load(path)
        max_ = mesh.vertices.max(axis=0)
        min_ = mesh.vertices.min(axis=0)
        new_vertices = mesh.vertices
        new_vertices[:,1] -= min_[1]    
        mesh = trimesh.Trimesh(vertices=new_vertices, faces=mesh.faces)
        #save
        mesh.export(path)
    else:
        mesh = trimesh.load(path)
        new_vertices = mesh.vertices
        new_vertices[:,1] -= min_[1]
        mesh = trimesh.Trimesh(vertices=new_vertices, faces=mesh.faces)
        #save
        mesh.export(path)
        
    # mesh = trimesh.load(path)
    # max_ = mesh.vertices.max(axis=0)
    # min_ = mesh.vertices.min(axis=0)
    # shift = (max_ + min_)/2
    # print(mesh.vertices.max(axis=0))
    # print(mesh.vertices.min(axis=0))
    # mesh_vertices = mesh.vertices
    # mesh_vertices[:,1] -= shift[1]
    # print(mesh_vertices.max(axis=0))
    # print(mesh_vertices.min(axis=0))
    # new_mesh = trimesh.Trimesh(vertices=mesh_vertices, faces=mesh.faces)
    # #export 
    # new_mesh.export(path)

In [ ]:
import trimesh
file_path = "/home/zqq/lagrangian_gaussian_splatting/Simulation/Codim-IPC/Projects/FEMShell/input/dress_knife/drape.obj"
mesh = trimesh.load(file_path)
vertices = mesh.vertices
print(mesh.faces)
mesh_stage = trimesh.load("/home/zqq/lagrangian_gaussian_splatting/Simulation/Codim-IPC/Projects/FEMShell/input/dress_knife/stage.obj")
length = mesh_stage.vertices.shape[0]
faces = mesh.faces
import numpy as np
#use top 1000 vertices to get submesh using trimesh
vertices = vertices[:length]
faces = []
for f in mesh.faces:
    if f[0] < length and f[1] < length and f[2] < length:
        faces.append(f)
submesh = trimesh.Trimesh(vertices=vertices, faces=faces)
submesh.show()
#save submesh
submesh.export("/home/zqq/lagrangian_gaussian_splatting/Simulation/Codim-IPC/Projects/FEMShell/input/dress_knife/submesh.obj")

In [ ]:
file_path = "/home/zqq/lagrangian_gaussian_splatting/Simulation/Codim-IPC/Projects/FEMShell/input/dress_knife/stage.obj"
mesh = trimesh.load(file_path)
print(mesh.vertices.shape)

In [9]:
import numpy as np
import trimesh
file_path = "/home/zqq/lagrangian_gaussian_splatting/Simulation/Codim-IPC/Projects/FEMShell/input/actorhq_smplx/shell0.obj"
mesh = trimesh.load(file_path)
file_path_cloth = "/home/zqq/lagrangian_gaussian_splatting/Simulation/Codim-IPC/Projects/FEMShell/input/a_skirt_new/dress.obj"
mesh_2 = trimesh.load(file_path_cloth)
vertices_cloth =  mesh_2.vertices
cloth_vertice_num = vertices_cloth.shape[0]
face_cloth = mesh_2.faces
vertices = mesh.vertices
faces = mesh.faces


import open3d
mesh_o3d = open3d.geometry.TriangleMesh()
mesh_o3d.vertices = open3d.utility.Vector3dVector(vertices_cloth)
mesh_o3d.triangles = open3d.utility.Vector3iVector(face_cloth)
triangle_clusters, cluster_n_triangles, cluster_area = mesh_o3d.cluster_connected_triangles()
max_cluster = np.argmax(cluster_area)
mesh_o3d.remove_triangles_by_mask(np.array(triangle_clusters) != max_cluster)
vertices_cloth = np.array(mesh_o3d.vertices)
face_cloth = np.array(mesh_o3d.triangles)

face_total = []
vertices_total = []
for elem in vertices_cloth:
    vertices_total.append(elem)
for elem in vertices:
    vertices_total.append(elem)

for elem in face_cloth:
    face_total.append([elm for elm in elem])
for elem in faces:
    face_total.append([i+cloth_vertice_num for i in elem])
total_mesh = trimesh.Trimesh(vertices=np.array(vertices_total), faces=np.array(face_total))
path = "/home/zqq/lagrangian_gaussian_splatting/Simulation/Codim-IPC/Projects/FEMShell/input/a_skirt_new/drape.obj"
total_mesh.export(path)

import trimesh 
mesh = trimesh.load(path)
min_ = mesh.vertices.min(axis=0)
max_ = mesh.vertices.max(axis=0)
mesh_vertices = mesh.vertices
mesh_vertices[:,1] -= min_[1]
mesh_vertices[:,1] -= 0.75
print(mesh_vertices.min(axis=0))
mesh = trimesh.Trimesh(vertices=mesh_vertices, faces=mesh.faces)
#export
mesh.export(path)

[-0.28251451 -0.75       -0.218252  ]


'# https://github.com/mikedh/trimesh\nv 0.04345000 -0.13529701 -0.16032700\nv 0.05206600 -0.13216501 -0.16930500\nv 0.06300500 -0.12937301 -0.17374400\nv 0.07418900 -0.12662801 -0.17728300\nv 0.08509900 -0.12379901 -0.18178100\nv 0.09497300 -0.12081101 -0.18879100\nv 0.09997600 -0.11790201 -0.20112100\nv 0.09429700 -0.11658701 -0.21346100\nv 0.08306300 -0.11661301 -0.21821400\nv 0.07113900 -0.11691901 -0.21817000\nv 0.05923200 -0.11724801 -0.21740900\nv 0.04731500 -0.11755701 -0.21686900\nv 0.03539300 -0.11785001 -0.21648800\nv 0.02346500 -0.11812701 -0.21627300\nv 0.01153800 -0.11838501 -0.21623500\nv -0.00038800 -0.11861801 -0.21643600\nv -0.01231100 -0.11883401 -0.21684100\nv -0.02422500 -0.11902901 -0.21747500\nv -0.03613500 -0.11920401 -0.21825200\nv -0.04802200 -0.11947101 -0.21714800\nv -0.05757500 -0.12040201 -0.20875600\nv -0.05794200 -0.12272401 -0.19490400\nv -0.05171100 -0.12517701 -0.18322500\nv -0.04374400 -0.12752301 -0.17309900\nv -0.03521400 -0.12979901 -0.16360900\nv 

In [ ]:
cloth_mesh = trimesh.Trimesh(vertices=vertices_cloth, faces=face_cloth)
path = "/home/zqq/lagrangian_gaussian_splatting/Simulation/Codim-IPC/Projects/FEMShell/input/dress_actorhq/550_submesh_processed.obj"
cloth_mesh.export(path)
import pymeshlab
ms = pymeshlab.MeshSet()
ms.load_new_mesh(path)
# not familiar with the crop API, but I'm sure it's doable
# now we generate UV map; there are a couple options here but this is a naive way
ms.compute_texcoord_by_function_per_vertex()
#export uv map
ms.save_current_mesh(path)


In [ ]:
# face = total_mesh.faces
# vertices = total_mesh.vertices
# mesh_new = trimesh.Trimesh(vertices=vertices, faces=face)

In [ ]:
total_mesh.export("/home/zqq/lagrangian_gaussian_splatting/Simulation/Codim-IPC/Projects/FEMShell/input/dress_actorhq/drape.obj")

In [ ]:
from pytorch3d.io import load_objs_as_meshes, save_obj
import torch
save_obj("/home/zqq/lagrangian_gaussian_splatting/Simulation/Codim-IPC/Projects/FEMShell/input/dress_actorhq/shell1.obj", torch.Tensor(total_mesh.vertices), torch.Tensor(total_mesh.faces))

In [ ]:
mesh_2.export("/home/zqq/lagrangian_gaussian_splatting/Simulation/Codim-IPC/Projects/FEMShell/input/dress_actorhq/shell_cloth.obj")

In [ ]:
face_cloth

In [ ]:
#save total_mesh
total_mesh.export("/home/zqq/lagrangian_gaussian_splatting/Simulation/Codim-IPC/Projects/FEMShell/input/dress_actorhq/shell_cloth.obj")

In [ ]:
face_total

In [ ]:
vertices.shape